In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split   #sklearn is one of many python APIs that exist
from sklearn.metrics import accuracy_score
import pickle
import pyarrow
import fastparquet


In [17]:
with open('trained_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [18]:
input_file = './troop_movements10m.csv'
dataset = pd.read_csv(input_file)
df = pd.DataFrame(dataset); df

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst
...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee


In [19]:
#df.replace({'unit_type': 'invalid_unit'}, {'unit_type': 'unknown'}); df
#df.replace('invalid_unit', 'unknown'); df
#df.replace({'unit_type': 'invalid_unit'}, value='unknown'); df
#df.replace(['invalid_unit'], ['unknown']); df
df.loc[df['unit_type'] == "invalid_unit", "unit_type"] = "unknown" ; df


,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst
...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee


In [20]:
df['location_x'].fillna(method='ffill', inplace=True); df
df['location_y'].fillna(method='ffill', inplace=True); df
#df['location_x'] =df['location_x'].fillna(method='ffill'); df

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst
...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee


In [21]:
df.to_parquet('troop_movements10m.parquet')

In [22]:
predicted_df = pd.read_parquet('troop_movements10m.parquet', engine='fastparquet'); predicted_df

#predictions = model.predict(predicted_df[['homeworld', 'unit_type']]); predictions

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-13 17:33:18,1,at-st,2.0,8.0,1,1,Glee Anselm
1,2023-06-13 17:33:17,2,tie_silencer,4.0,4.0,0,1,Trandosha
2,2023-06-13 17:33:16,3,at-at,0.0,3.0,6,1,Corellia
3,2023-06-13 17:33:15,4,tie_silencer,6.0,1.0,6,9,Shili
4,2023-06-13 17:33:14,5,tie_fighter,0.0,4.0,9,6,Muunilinst
...,...,...,...,...,...,...,...,...
9999995,2023-02-18 10:26:20,9999996,resistance_soldier,6.0,6.0,1,3,Troiken
9999996,2023-02-18 10:26:19,9999997,tie_silencer,8.0,6.0,2,0,Kashyyyk
9999997,2023-02-18 10:26:18,9999998,tie_fighter,7.0,7.0,6,4,Kashyyyk
9999998,2023-02-18 10:26:17,9999999,tie_fighter,6.0,6.0,8,8,Kalee


In [23]:
test_df = predicted_df[['homeworld', 'unit_type']]
X_encoded = pd.get_dummies(test_df)
#y_encoded = pd.get_dummies(y)
y_pred = model.predict(X_encoded)
print(y_pred)


[False False False ... False False False]


In [27]:
#print(type(y_pred))
test_df['predictions'] = y_pred
test_df

C:\Users\wasadmin\AppData\Local\Temp\ipykernel_4828\155968281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predictions'] = y_pred


,homeworld,unit_type,predictions
0,Glee Anselm,at-st,False
1,Trandosha,tie_silencer,False
2,Corellia,at-at,False
3,Shili,tie_silencer,False
4,Muunilinst,tie_fighter,False
...,...,...,...
9999995,Troiken,resistance_soldier,True
9999996,Kashyyyk,tie_silencer,False
9999997,Kashyyyk,tie_fighter,False
9999998,Kalee,tie_fighter,False


In [25]:
#predicted_df['prediction']=pd.Series(y_pred); predicted_df
#predicted_df['prediction'] = pd.DataFrame(y_pred, columns=['prediction']); predicted_df
#test_column = pd.DataFrame(y_pred, columns=['prediction']); test_column
test_column = pd.DataFrame(y_pred); test_column

,0
0,False
1,False
2,False
3,False
4,False
...,...
9999995,True
9999996,False
9999997,False
9999998,False
